###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

## Model Selection
Model selection is choosing between different complexity or flexibility to fit/explain your data.
#### Complexity Controlling Parameters
* Not used to fit the data
* Usually called "hyperparameters" (i.e., in Bayesian models parameters of the prior for instance)

#### Selecting best complexity parameters
Selecting the best complexity parameters usually consist of exploring different combinations of the hyperparameters to explain your data. More technically this search consist of:
* An estimator (regressor or classifier )
* A parameter space
* A method for searching or sampling candidates
* A cross-validation scheme
* A score function

In [1]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

### Dataset 1
Performs model selection of the following hyperparameters applied to the bank dataset (customers leaving):
* Penalty (l1 or l2)
* C (Inverse of regularization)

This is perform using a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).


In [2]:
# Loads dataset from file
dataset = pd.read_csv('E:/datasets/classification/bank_exiting.csv')
predictors = dataset.iloc[:,3:-1].values
responses = dataset.iloc[:,-1].values
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Encoding the gender feature
print(dataset['Gender'].unique())
print(predictors[1,:])
encoder = LabelEncoder()
predictors[:,2] = encoder.fit_transform(predictors[:,2]) # only 0 or 1 after this (just one column needed)
print(predictors[1,:])

['Female' 'Male']
[608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
[608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]


In [4]:
# Encoding the country feature
print(predictors[1,:])
ct = ColumnTransformer([('country_category', OneHotEncoder(categories='auto', drop='first'),[1])], remainder='passthrough')
predictors = ct.fit_transform(predictors)
print(predictors[1,:])

[608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
[0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


In [5]:
# Splits intro training/holdout
X, X_holdout, y, y_holdout = train_test_split(predictors, responses, test_size=0.2, stratify=responses)

In [6]:
# This step should not be here since we will leak info into the validation sets!!!
ct = ColumnTransformer([('normalizer', StandardScaler(), [2,4,5,6,7,10])], remainder='passthrough')
X = ct.fit_transform(X)
X_holdout = ct.transform(X_holdout)
X[0,:]

array([-1.6636522716407491, -1.2306938563643364, -0.3437761243945,
       0.414489509180079, -0.910289944180706, 0.7869945564393298, 1.0,
       0.0, 1, 1, 1], dtype=object)

In [7]:
# Defines grid
C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
penalty_choices = ['l1','l2']
hyperparams = [{
    'penalty': penalty_choices,
    'C': C_choices
}]
# Defines model
mdl = LogisticRegression(max_iter=200)
# Performs grid search using CV
validator = GridSearchCV(mdl, cv=3, param_grid=hyperparams, scoring='f1', n_jobs=-1, verbose=2)
validator.fit(X,y)
utils.report_search(validator.cv_results_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Model with rank: 1
Mean validation score: 0.306 (std: 0.019)
Parameters: {'C': 1, 'penalty': 'l2'}

Model with rank: 2
Mean validation score: 0.305 (std: 0.019)
Parameters: {'C': 0.7, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.305 (std: 0.018)
Parameters: {'C': 0.5, 'penalty': 'l2'}


[Parallel(n_jobs=-1)]: Done  32 out of  36 | elapsed:    1.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    1.7s finished


In [8]:
# Selects best configuration after search
best = validator.best_estimator_

In [9]:
# Retrains the model now on the available dataset (without validation/dev set). This step is optional!!!
best.fit(X, y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# Training performance
y_pred = best.predict(X)
utils.report_classification(y, y_pred, title='Train')

Train (Metrics): 

Accuracy: 0.81
F1 Score: 0.31
Recall: 0.21
Precision: 0.60

Confusion Matrix:
 [[6138  232]
 [1288  342]]


In [11]:
# Holdout performance
y_pred = best.predict(X_holdout)
utils.report_classification(y_holdout, y_pred, title='Holdout')

Holdout (Metrics): 

Accuracy: 0.81
F1 Score: 0.33
Recall: 0.23
Precision: 0.59

Confusion Matrix:
 [[1528   65]
 [ 314   93]]


### Dataset 2
Performs model selection of the following hyperparameters applied to the wine dataset:
* Penalty (l1 or l2)
* C (Inverse of regularization)
* Max # of iterations

This is perform using a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Notice that no standarization is applied in this case

In [12]:
# Load dataset
dataset = datasets.load_wine()
predictors = dataset.data
responses = dataset.target
X, X_holdout, y, y_holdout = train_test_split(predictors, responses, test_size=0.2, stratify=responses)

In [13]:
# Defines grid
C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
iter_choices = [10, 20, 50, 100, 300]
hyperparams = [{
    'penalty': ['l1'],
    'C': C_choices,
    'max_iter': iter_choices,
    'solver': ['saga']
},    
{
    'penalty': ['l2'],
    'C': C_choices,
    'max_iter': iter_choices,
    'solver': ['lbfgs']
}]
# Defines model
mdl = LogisticRegression(multi_class='multinomial')
# Performs grid search
validator = GridSearchCV(mdl, cv=5, param_grid=hyperparams, scoring='f1_macro', n_jobs=-1, verbose=2)
validator.fit(X,y)
utils.report_search(validator.cv_results_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s



Model with rank: 1
Mean validation score: 0.944 (std: 0.028)
Parameters: {'C': 1, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}

Model with rank: 2
Mean validation score: 0.944 (std: 0.028)
Parameters: {'C': 0.7, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}

Model with rank: 3
Mean validation score: 0.944 (std: 0.028)
Parameters: {'C': 0.5, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.8s finished


In [14]:
# Selects best configuration after search
best = validator.best_estimator_

In [15]:
# Retrains the model now on the available dataset (without validation/dev set). This step is optional!!!
best.fit(X, y)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
# Training performance
y_pred = best.predict(X)
utils.report_classification(y, y_pred, avg='macro', title='Train')

Train (Metrics): 

Accuracy: 0.96
F1 Score: 0.97
Recall: 0.97
Precision: 0.97

Confusion Matrix:
 [[45  2  0]
 [ 1 55  1]
 [ 0  1 37]]


In [17]:
# Holdout performance
y_pred = best.predict(X_holdout)
utils.report_classification(y_holdout, y_pred, avg='macro', title='Holdout')

Holdout (Metrics): 

Accuracy: 1.00
F1 Score: 1.00
Recall: 1.00
Precision: 1.00

Confusion Matrix:
 [[12  0  0]
 [ 0 14  0]
 [ 0  0 10]]


### Dataset 3
Performs model selection of the following hyperparameters applied to the wine dataset:
* Penalty (l1 or l2)
* C (Inverse of regularization)
* Max # of iterations

This is perform using a [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Since we want to apply standarization, we must use a [Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline) to correctly standarize and evaluate the models during the cross-validation.

In [18]:
# Load dataset
dataset = datasets.load_wine()
predictors = dataset.data
responses = dataset.target
X, X_holdout, y, y_holdout = train_test_split(predictors, responses, test_size=0.2)

In [19]:
# Defines pipeline
sc = StandardScaler()
clf = LogisticRegression(multi_class='ovr')
estimators = [('normalizer', sc), ('classifier', clf)]
pipe = Pipeline(estimators)
# Defines grid
C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
hyperparams = [{
    'classifier__penalty': ['l1'],
    'classifier__C': C_choices,
    'classifier__max_iter': iter_choices,
    'classifier__solver': ['saga']
},    
{
    'classifier__penalty': ['l2'],
    'classifier__C': C_choices,
    'classifier__max_iter': iter_choices,
    'classifier__solver': ['lbfgs']
}]
# Performs grid search using CV
validator = GridSearchCV(pipe, cv=5, param_grid=hyperparams, scoring='accuracy', n_jobs=-1, verbose=1)
validator.fit(X,y)
utils.report_search(validator.cv_results_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s



Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 1, 'classifier__max_iter': 10, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 2
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 1, 'classifier__max_iter': 20, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 3
Mean validation score: 0.979 (std: 0.028)
Parameters: {'classifier__C': 1, 'classifier__max_iter': 300, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.4s finished


In [20]:
# Selects best configuration after search
best = validator.best_estimator_

In [21]:
# Retrains the model now on the available dataset (without validation/dev set). This step is optional!!!
best.fit(X, y)

Pipeline(memory=None,
         steps=[('normalizer',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=10,
                                    multi_class='ovr', n_jobs=None,
                                    penalty='l1', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [22]:
# Training performance
y_pred = best.predict(X)
utils.report_classification(y, y_pred, avg='macro', title='Train')

Train (Metrics): 

Accuracy: 1.00
F1 Score: 1.00
Recall: 1.00
Precision: 1.00

Confusion Matrix:
 [[48  0  0]
 [ 0 53  0]
 [ 0  0 41]]


In [23]:
# Holdout performance
y_pred = best.predict(X_holdout)
utils.report_classification(y_holdout, y_pred, avg='macro', title='Holdout')

Holdout (Metrics): 

Accuracy: 0.97
F1 Score: 0.98
Recall: 0.98
Precision: 0.97

Confusion Matrix:
 [[11  0  0]
 [ 1 17  0]
 [ 0  0  7]]
